In [5]:
import os
import numpy as np
import pandas as pd
from PIL import Image

from google.cloud import storage

from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger, LearningRateScheduler
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, Conv2DTranspose, Concatenate, ZeroPadding2D, Dropout

/Users/Alenka/.pyenv/versions/3.10.6/envs/GlobalGreening/lib/python3.10/site-packages/tensorflow_io-0.32.0-py3.10-macosx-13.3-arm64.egg/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/Users/Alenka/.pyenv/versions/3.10.6/envs/GlobalGreening/lib/python3.10/site-packages/tensorflow_io-0.32.0-py3.10-macosx-13.3-arm64.egg/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ["[Errno 2] The file to load file system plugin from does not exist.: '/Users/Alenka/.pyenv/versions/3.10.6/envs/GlobalGreening/lib/python3.10/site-packages/tensorflow_io-0.32.0-py3.10-macosx-13.3-arm64.egg/tensorflow_io/python/ops/libtensorflow_io_plugins.so'"]
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/Users/Alenka/.pyenv/versions/3.10.6/envs/GlobalGreening/lib/python3.10/site-packages/tensorflow_io-0.32.0-py3.10-macosx-13.3-arm64.egg/tensorflow_io/python/ops/__init__

In [68]:
# parameters
DATA_SIZE=15416 
TRIAL_SIZE=1000     # after running it once change to a larger number (up to 15000)
LOAD_CHUNK_SIZE=47 # this number so that all batches are same size
DATA_RUN = 'local_VM' #  select from these options:'gcp' local_VM' , 'local' 

In [7]:
!pwd

/Users/Alenka/code/Alastair908/GlobalGreening/notebooks


In [77]:
# this structure has to be changed here or in separate functions for loading 

VM_dataset_folder =  "../raw_data"   
VM_images_dir = "images_trial_run"
VM_masks_dir = "masks"
VM_output_folder = "../training_outputs"  

## Functions and variables to load

In [78]:
def list_image_filenames(dataset_root_folder, images_dir):
    """Lists all the image files in the folder"""
    images_directory = f'{dataset_root_folder}/{images_dir}'
    print(f'loading file names from {images_directory}')
    file_list = os.listdir(images_directory)
    image_file_list = []
    for file_name in file_list:
        if file_name.endswith('.png'):
            image_file_list.append(file_name)
    image_files = np.sort(image_file_list)
    return image_files

def get_image_geo_locations(dataset_root_folder, images_dir):
    
    # generating file names from the directory
    image_files = list_image_filenames(dataset_root_folder, images_dir)
    file_names = np.char.rstrip(image_files, '.png')
    file_names = np.char.split(file_names, '_')
    print(f'sample images file names {image_files[:3]} and amount of images in the directory {len(image_files)}')
    print(f'sample split file names {file_names[:3]}, and check that same lenght of lists {len(file_names)}')
    # creating df with information about longitude, latitude (can also be used to load the images)
    image_geo_locations = np.zeros((len(file_names),2))
    image_geo_locations = pd.DataFrame(image_geo_locations, columns=['latitude', 'longitude'])
    for image_type in ['latitude', 'longitude']:
        for i in range(len(file_names)):
            file = file_names[i]
            if image_type == 'latitude':
                text = file[0]
                image_number = ''.join(num for num in text if num.isdigit())
                latitude = file[1].strip('-')
    #            print(f'latitude is {latitude}')
                image_geo_locations.at[int(image_number),'latitude'] = latitude
            elif image_type == 'longitude':
                text = file[0]
                image_number = ''.join(num for num in text if num.isdigit())
                longitude = file[2]
    #            print(f'longitude is {longitude}')
                image_geo_locations.at[int(image_number),'longitude'] = longitude
    return image_geo_locations

def load_images(dataset_root_folder, images_dir, load_range_images, image_geo_locations):
    
    # loading images into the list (new code so the files are loaded in correct order (by index))
    images_dataset = []
    
    for i in load_range_images:
        image_file =f'image{i}_-{image_geo_locations.iat[i,0]}_{image_geo_locations.iat[i,1]}.png'
        path_image = f'{dataset_root_folder}/{images_dir}/{image_file}'
        # print(f'loading the image from these file {path_image}')
        image = Image.open(path_image)

        if np.asarray(image).shape[2] >3:
            image = image.convert('RGB')
        image = np.asarray(image)
        image = image/255.  # includes pre-processing
        images_dataset.append(image)
        # print(f'appended image of size {image.shape}')
        
    images_dataset = np.array(images_dataset)
    
    return images_dataset

In [79]:
# change the folders here or call variables defined above

image_geo_locations = get_image_geo_locations(
    "../raw_data", # VM_dataset_folder
     "images_trial_run"  # VM_images_dir
)

loading file names from ../raw_data/images_trial_run
sample images file names ['image0_-109.0_37.0.png' 'image100_-108.96_37.26.png'
 'image101_-108.96_37.3.png'] and amount of images in the directory 500
sample split file names [list(['image0', '-109.0', '37.0']) list(['image100', '-108.96', '37.26'])
 list(['image101', '-108.96', '37.3'])], and check that same lenght of lists 500


In [12]:
image_geo_locations.head()

,latitude,longitude
0,109.0,37.0
1,109.0,37.04
2,109.0,37.09
3,109.0,37.13
4,109.0,37.17


In [13]:
image_geo_locations.shape

(500, 2)

## Compile and Load Model

In [14]:
def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_inception_resnetv2_unet(input_shape):
    """ Input """
    inputs = Input(input_shape)

    """ Pre-trained InceptionResNetV2 Model """
    encoder = InceptionResNetV2(include_top=False, weights="imagenet", input_tensor=inputs)

    """ Encoder """
    s1 = encoder.get_layer("input_1").output           ## (512 x 512)

    s2 = encoder.get_layer("activation").output        ## (255 x 255)
    s2 = ZeroPadding2D(( (1, 0), (1, 0) ))(s2)         ## (256 x 256)

    s3 = encoder.get_layer("activation_3").output      ## (126 x 126)
    s3 = ZeroPadding2D((1, 1))(s3)                     ## (128 x 128)

    s4 = encoder.get_layer("activation_74").output      ## (61 x 61)
    s4 = ZeroPadding2D(( (2, 1),(2, 1) ))(s4)           ## (64 x 64)

    """ Bridge """
    b1 = encoder.get_layer("activation_161").output     ## (30 x 30)
    b1 = ZeroPadding2D((1, 1))(b1)                      ## (32 x 32)

    """ Decoder """
    d1 = decoder_block(b1, s4, 512)                     ## (64 x 64)
    d2 = decoder_block(d1, s3, 256)                     ## (128 x 128)
    d3 = decoder_block(d2, s2, 128)                     ## (256 x 256)
    d4 = decoder_block(d3, s1, 64)                      ## (512 x 512)
    
    """ Output """
    dropout = Dropout(0.3)(d4)
    outputs = Conv2D(11, 1, padding="same", activation="softmax")(dropout)

    model = Model(inputs, outputs, name="InceptionResNetV2-UNet")
    return model

In [15]:
K.clear_session()

def dice_coef(y_true, y_pred):
    return (2. * K.sum(y_true * y_pred) + 1.) / (K.sum(y_true) + K.sum(y_pred) + 1.)

model = build_inception_resnetv2_unet(input_shape = (512, 512, 3))
model.compile(optimizer=Adam(lr = 0.0001), loss='categorical_crossentropy', metrics=[dice_coef, "accuracy"])
model.summary()

Model: "InceptionResNetV2-UNet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 255, 255, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 255, 255, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                            

                                                                                                  
 conv2d_10 (Conv2D)             (None, 61, 61, 96)   82944       ['activation_9[0][0]']           
                                                                                                  
 conv2d_11 (Conv2D)             (None, 61, 61, 64)   12288       ['average_pooling2d[0][0]']      
                                                                                                  
 batch_normalization_5 (BatchNo  (None, 61, 61, 96)  288         ['conv2d_5[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 batch_normalization_7 (BatchNo  (None, 61, 61, 64)  192         ['conv2d_7[0][0]']               
 rmalization)                                                                                     
          

 conv2d_21 (Conv2D)             (None, 61, 61, 32)   10240       ['block35_1_ac[0][0]']           
                                                                                                  
 batch_normalization_21 (BatchN  (None, 61, 61, 32)  96          ['conv2d_21[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_21 (Activation)     (None, 61, 61, 32)   0           ['batch_normalization_21[0][0]'] 
                                                                                                  
 conv2d_19 (Conv2D)             (None, 61, 61, 32)   10240       ['block35_1_ac[0][0]']           
                                                                                                  
 conv2d_22 (Conv2D)             (None, 61, 61, 48)   13824       ['activation_21[0][0]']          
          

 batch_normalization_26 (BatchN  (None, 61, 61, 32)  96          ['conv2d_26[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_29 (BatchN  (None, 61, 61, 64)  192         ['conv2d_29[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_24 (Activation)     (None, 61, 61, 32)   0           ['batch_normalization_24[0][0]'] 
                                                                                                  
 activation_26 (Activation)     (None, 61, 61, 32)   0           ['batch_normalization_26[0][0]'] 
                                                                                                  
 activatio

 conv2d_37 (Conv2D)             (None, 61, 61, 32)   10240       ['block35_4_ac[0][0]']           
                                                                                                  
 conv2d_40 (Conv2D)             (None, 61, 61, 48)   13824       ['activation_39[0][0]']          
                                                                                                  
 batch_normalization_37 (BatchN  (None, 61, 61, 32)  96          ['conv2d_37[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_40 (BatchN  (None, 61, 61, 48)  144         ['conv2d_40[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activatio

...

                                                                                                  
 batch_normalization_134 (Batch  (None, 30, 30, 160)  480        ['conv2d_134[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_134 (Activation)    (None, 30, 30, 160)  0           ['batch_normalization_134[0][0]']
                                                                                                  
 conv2d_132 (Conv2D)            (None, 30, 30, 192)  208896      ['block17_14_ac[0][0]']          
                                                                                                  
 conv2d_135 (Conv2D)            (None, 30, 30, 192)  215040      ['activation_134[0][0]']         
                                                                                                  
 batch_nor

                                                                                                  
 activation_142 (Activation)    (None, 30, 30, 160)  0           ['batch_normalization_142[0][0]']
                                                                                                  
 conv2d_140 (Conv2D)            (None, 30, 30, 192)  208896      ['block17_16_ac[0][0]']          
                                                                                                  
 conv2d_143 (Conv2D)            (None, 30, 30, 192)  215040      ['activation_142[0][0]']         
                                                                                                  
 batch_normalization_140 (Batch  (None, 30, 30, 192)  576        ['conv2d_140[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_nor

 conv2d_148 (Conv2D)            (None, 30, 30, 192)  208896      ['block17_18_ac[0][0]']          
                                                                                                  
 conv2d_151 (Conv2D)            (None, 30, 30, 192)  215040      ['activation_150[0][0]']         
                                                                                                  
 batch_normalization_148 (Batch  (None, 30, 30, 192)  576        ['conv2d_148[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_151 (Batch  (None, 30, 30, 192)  576        ['conv2d_151[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activatio

 conv2d_transpose (Conv2DTransp  (None, 64, 64, 512)  590336     ['zero_padding2d_3[0][0]']       
 ose)                                                                                             
                                                                                                  
 zero_padding2d_2 (ZeroPadding2  (None, 64, 64, 256)  0          ['activation_74[0][0]']          
 D)                                                                                               
                                                                                                  
 concatenate (Concatenate)      (None, 64, 64, 768)  0           ['conv2d_transpose[0][0]',       
                                                                  'zero_padding2d_2[0][0]']       
                                                                                                  
 conv2d_203 (Conv2D)            (None, 64, 64, 512)  3539456     ['concatenate[0][0]']            
          

 conv2d_209 (Conv2D)            (None, 512, 512, 64  38656       ['concatenate_3[0][0]']          
                                )                                                                 
                                                                                                  
 batch_normalization_209 (Batch  (None, 512, 512, 64  256        ['conv2d_209[0][0]']             
 Normalization)                 )                                                                 
                                                                                                  
 activation_209 (Activation)    (None, 512, 512, 64  0           ['batch_normalization_209[0][0]']
                                )                                                                 
                                                                                                  
 conv2d_210 (Conv2D)            (None, 512, 512, 64  36928       ['activation_209[0][0]']         
          

In [16]:
# replace with your local model path
model_path = "../training_outputs/models/20230611-082522_InceptionResNetV2-UNet.h5"

In [17]:
model.load_weights(model_path)

## "FOR LOOP" to generate single mask predictions (512,) and save them into folder

In [18]:
id2single = {0: 1, 
             1: 1, 
             2: 0, 
             3: 0, 
             4: 0, 
             5: 1, 
             6: 0, 
             7: 0, 
             8: 1, 
             9: 1, 
             10: 1}

def onehot_to_single_new(onehot):
    '''Function to decode encoded mask labels
        Inputs: 
            onehot - one hot encoded image matrix (height x width x num_classes)
            colormap - dictionary of color to label id
        Output: Decoded RGB image (height x width x 3) 
    '''
    single_layer = np.argmax(onehot, axis=-1)
    output = np.zeros( onehot.shape[:2] )
    for k in id2single.keys():
        output[single_layer==k] = id2single[k]
    return np.uint8(output)

In [19]:
# old function just in case it goes wrong

def onehot_to_single(onehot, colormap = id2single):
    '''Function to decode encoded mask labels
        Inputs: 
            onehot - one hot encoded image matrix (height x width x num_classes)
            colormap - dictionary of color to label id
        Output: Decoded RGB image (height x width x 3) 
    '''
    single_layer = np.argmax(onehot, axis=-1)
    output = np.zeros( onehot.shape[:2]+(1,) )
    for k in colormap.keys():
        output[single_layer==k] = colormap[k]
    return np.uint8(output)

In [65]:
def predict_and_save_masks(dataset_root_folder, images_dir, dataset_size, image_geo_locations, output_folder):
    for i in range(dataset_size):
        images_dataset = load_images(dataset_root_folder,
                                     images_dir,
                                     range(i,i + 1),
                                    image_geo_locations)

        y_pred = model.predict(images_dataset)

        mask_pred = onehot_to_single_new(y_pred[0,:,:,:])

        mask_pred_path = f'{output_folder}/pred_masks/pred_mask{i}_-{image_geo_locations.iat[i,0]}_{image_geo_locations.iat[i,1]}.npy'

        np.save(mask_pred_path,mask_pred)
    print(f'completed and saved all predicted masks')

In [72]:
number_of_batch_loading = int(DATA_SIZE / LOAD_CHUNK_SIZE)
number_of_batch_loading

328

In [75]:
# eventually if it works, the range should be the variable number_of_batch_loading (or 328)

for i in range(1):
    print(f'start batch {i+1} of total {number_of_batch_loading} batches to predict {LOAD_CHUNK_SIZE} masks')
   
    predict_and_save_masks(
        "../raw_data",
        "images_trial_run",
        LOAD_CHUNK_SIZE,
        image_geo_locations,
        "../training_outputs"
    )

start batch 1 of total 328 batches 
1/1 [==============================] - 1s 687ms/step
completed and saved all predicted masks


### Testing code

In [ ]:
#for i in range(15416):
    images_dataset = load_images("/home/jupyter",
                                 "zoomed_photos/zoomed_photos",
                                 range(i,i + 1))
    
    y_pred = model.predict(images_dataset)
    
    mask_pred = onehot_to_single_new(y_pred)
    
    mask_pred_path = f'pred_masks/pred_mask{i}_-{image_geo_locations.iat[i,0]}_{image_geo_locations.iat[i,1]}.npy'
    
    np.save(mask_pred_path,mask_pred)

In [ ]:
#load_range_images = range(8050,8075)  # this is define as from to (non-inclusive to, so range(200,500), means from 200-499

images_dataset = load_images(
    "../raw_data",  # VM_dataset_folder 
    "images_trial_run",  # VM_images_dir
    range(20,40),
    image_geo_locations
)

In [ ]:
images_dataset.shape

In [76]:
i =3
print(f'start batch {i+1} of total {number_of_batch_loading} batches to predict {LOAD_CHUNK_SIZE} masks')

start batch 4 of total 328 batches to predict 47 masks


In [ ]:
# pred_all = model.predict(images_dataset)

In [ ]:
for i in range(1):
    images_dataset = load_images("../raw_data",
                                 "images_trial_run", 
                                 range(i,i + 1),
                                image_geo_locations)
    
    y_pred = model.predict(images_dataset)
    mask_pred = onehot_to_single_new(y_pred[0,:,:,:])
    
    mask_pred_path = f'{output_folder}/pred_masks/pred_mask{i}_-{image_geo_locations.iat[i,0]}_{image_geo_locations.iat[i,1]}.npy'
 
    np.save(mask_pred_path,mask_pred)

In [38]:
i =100
output_folder = "../training_outputs" 
path_mask_sample = "../raw_data/masks/mask100_-108.96_37.26.npy"
mask_pred_path_sample = f'{output_folder}/pred_masks/pred_mask{i}_-{image_geo_locations.iat[i,0]}_{image_geo_locations.iat[i,1]}.npy'
mask_pred_path_sample

'../training_outputs/pred_masks/pred_mask100_-108.96_37.26.npy'

In [39]:
mask_sample = np.load(path_mask_sample)

In [53]:
np.save(mask_pred_path_sample,output_new)

In [40]:
mask_sample.shape

(512, 512, 3)

In [41]:
# dictionary with class names + dummy rbg values
labels_dict = {"classes": [
    {"title": "Tree cover", "r": 0 , "g": 0 , "b": 0 }, 
    {"title": "Shrubland", "r": 0, "g": 0, "b": 0 }, 
    {"title": "Grassland", "r": 0, "g": 0, "b": 0 }, 
    {"title": "Cropland", "r": 0, "g": 0, "b": 0 }, 
    {"title": "Built-up", "r": 0, "g": 0, "b": 0 }, 
    {"title": "Bare, sparse vegetation", "r": 0, "g": 0, "b": 0 },
    {"title": "Snow and ice", "r": 0, "g": 0, "b": 0 },
    {"title": "Permanent water bodies", "r": 0, "g": 0, "b": 0 },
    {"title": "Herbaceous wetland", "r": 0, "g": 0, "b": 0 },
    {"title": "Mangroves", "r": 0, "g": 0, "b": 0 },
    {"title": "Moss and lichen", "r": 0, "g": 0, "b": 0 }
    ]}

# loading correct rgb values from hex_color list based on ESA  
hex_colors_list = ['#006400', '#ffbb22', '#ffff4c', '#f096ff', '#fa0000',
                    '#b4b4b4', '#f0f0f0', '#0064c8', '#0096a0', '#00cf75', '#fae6a0']


# preparing label df with all information: classes, rgb values

def prepare_labels(labels_dict, hex_colors_list):

    labels_dict_df = pd.DataFrame(labels_dict['classes'])
    
    for i in range(len(hex_colors_list)):
        color = hex_colors_list[i].lstrip('#')
        r = int(color[0:2],16)
        g = int(color[2:4],16)
        b = int(color[4:6],16)
        labels_dict_df.at[i,'r'] = r
        labels_dict_df.at[i,'g'] = g
        labels_dict_df.at[i,'b'] = b
    
    return labels_dict_df

# preparing label codes (just rgb values in order of classes)

def prepare_label_codes(labels_dict, hex_colors_list):
    labels_dict_df = prepare_labels(labels_dict, hex_colors_list)
    
    label_codes = []
    r= np.asarray(labels_dict_df.r)
    g= np.asarray(labels_dict_df.g)
    b= np.asarray(labels_dict_df.b)

    for i in range(len(labels_dict_df)):
        label_codes.append(tuple([r[i], g[i], b[i]]))
    return label_codes

# preparing label names (just class names)

def prepare_label_names(labels_dict, hex_colors_list):
    labels_dict_df = prepare_labels(labels_dict, hex_colors_list)   
    label_names= list(labels_dict_df.title)
    return label_names

label_codes = prepare_label_codes(labels_dict, hex_colors_list)
label_names = prepare_label_names(labels_dict, hex_colors_list)

code2id = {v:k for k,v in enumerate(label_codes)}
id2code = {k:v for k,v in enumerate(label_codes)}

name2id = {v:k for k,v in enumerate(label_names)}
id2name = {k:v for k,v in enumerate(label_names)}

def rgb_to_onehot(rgb_mask_image, colormap = id2code):
    '''Function to one hot encode RGB mask labels
        Inputs: 
            rgb_image - image matrix (eg. 512 x 512 x 3 dimension numpy ndarray)
            colormap - dictionary of color to label id
        Output: One hot encoded image of dimensions (height x width x num_classes) where num_classes = len(colormap)
    '''
    num_classes = len(colormap)
    
    # shape prepared for image size and channels = num of classes (instead of 3 RGB colors)
    shape = rgb_mask_image.shape[:2]+(num_classes,)
    
    # encoded_image prepare array with right shaoe 
    encoded_mask = np.zeros( shape, dtype=np.int8 )
    for i, cls in enumerate(colormap):
        # image.reshape flattens and keeps 3 channels, then checks which pixels same as color in colormap
        # then change back to image size for each of 6 channels (based on colormap)
        encoded_mask[:,:,i] = np.all(rgb_mask_image.reshape( (-1,3) ) == colormap[i], axis=1).reshape(shape[:2])

    return encoded_mask

In [42]:
onehot_mask_sample = rgb_to_onehot(mask_sample, colormap = id2code)

In [43]:
onehot_mask_sample.shape

(512, 512, 11)

In [44]:
single_layer = np.argmax(onehot_mask_sample, axis=-1)
single_layer.shape

(512, 512)

In [45]:
onehot_mask_sample.shape[:2]

(512, 512)

In [46]:
output = np.zeros(onehot_mask_sample.shape[:2])
output.shape

(512, 512)

In [47]:
for k in id2single.keys():
    output[single_layer==k] = id2single[k]
output = np.uint8(output)

output.shape

(512, 512)

In [48]:
output

array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]], dtype=uint8)

In [49]:
output_old = onehot_to_single(onehot_mask_sample, colormap = id2single)
print(output_old.shape)
output_old

(512, 512, 1)


array([[[1],
        [1],
        [1],
        ...,
        [1],
        [1],
        [1]],

       [[1],
        [1],
        [1],
        ...,
        [1],
        [1],
        [1]],

       [[1],
        [1],
        [1],
        ...,
        [1],
        [1],
        [1]],

       ...,

       [[1],
        [1],
        [1],
        ...,
        [1],
        [1],
        [1]],

       [[1],
        [1],
        [1],
        ...,
        [1],
        [1],
        [1]],

       [[1],
        [1],
        [1],
        ...,
        [1],
        [1],
        [1]]], dtype=uint8)

In [50]:
output_new = onehot_to_single_new(onehot_mask_sample)
print(output_new.shape)
output_new

(512, 512)


array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]], dtype=uint8)

In [51]:
output_new2 = onehot_to_single(onehot_mask_sample, colormap = id2single)
print(output_new2.shape)
output_new2[,:,:].shape

SyntaxError: invalid syntax (1038929380.py, line 3)